<a href="https://colab.research.google.com/github/bryaanabraham/Wandering_Guardian/blob/main/ML%20Implementation/ML_Navigation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [250]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import folium
import random
from geopy.distance import geodesic

In [251]:
coords = pd.read_csv("training_dataset.csv")
coords.head()

,longitude,latitude,path_id
0,12.821062,80.044729,1.0
1,12.821219,80.044743,1.0
2,12.821479,80.044736,1.0
3,12.821561,80.044743,1.0
4,12.821636,80.044743,1.0


In [252]:
path = []
for i in range(1,len(coords['path_id'].unique())):
    path.append(coords[coords['path_id'] == i])

In [253]:
path[0].head()

,longitude,latitude,path_id
0,12.821062,80.044729,1.0
1,12.821219,80.044743,1.0
2,12.821479,80.044736,1.0
3,12.821561,80.044743,1.0
4,12.821636,80.044743,1.0


In [254]:
# Deviation parameters
min_deviation_distance = -0.00004
max_deviation_distance = 0.00004
#deviation is set to zero for now so elevation remains constant
max_deviation_angle = 0

# Generate random deviation distances and angles for each point in the DataFrame
num_points = len(coords)
random_deviation_distances = np.random.uniform(min_deviation_distance, max_deviation_distance, num_points)
random_deviation_angles = np.random.uniform(0, max_deviation_angle, num_points)

# Convert random angles to radians
random_deviation_angles_radians = np.radians(random_deviation_angles)

# Calculate deviation in latitude and longitudet
deviation_distance_lat = random_deviation_distances * np.cos(random_deviation_angles_radians)
deviation_distance_lon = random_deviation_distances * np.sin(random_deviation_angles_radians)

decoy_coords = coords.copy()
decoy_coords['latitude'] += deviation_distance_lat
decoy_coords['longitude'] += deviation_distance_lon

decoy_coords.head()

,longitude,latitude,path_id
0,12.821062,80.044731,1.0
1,12.821219,80.044739,1.0
2,12.821479,80.044696,1.0
3,12.821561,80.044754,1.0
4,12.821636,80.044749,1.0


In [255]:
decoy_coords.head()

,longitude,latitude,path_id
0,12.821062,80.044731,1.0
1,12.821219,80.044739,1.0
2,12.821479,80.044696,1.0
3,12.821561,80.044754,1.0
4,12.821636,80.044749,1.0


In [256]:
print('coords keys:', coords.keys())
print('decoy_coords keys:', decoy_coords.keys())

coords keys: Index(['longitude', 'latitude', 'path_id'], dtype='object')
decoy_coords keys: Index(['longitude', 'latitude', 'path_id'], dtype='object')


In [257]:
decoy_path = []
for i in range(1, len(decoy_coords['path_id'].unique())):
    decoy_path.append(decoy_coords[decoy_coords['path_id'] == i])

decoy_path[0].head()

,longitude,latitude,path_id
0,12.821062,80.044731,1.0
1,12.821219,80.044739,1.0
2,12.821479,80.044696,1.0
3,12.821561,80.044754,1.0
4,12.821636,80.044749,1.0


In [258]:
maps = []

colors = ['red','blue','green','yellow','orange','purple','pink','cyan','black','brown','maroon','navy','teal','olive','lime','aqua','silver','gold']

max_lat = max(coords['longitude'])
min_lat = min(coords['longitude'])
min_lon = min(coords['latitude'])
max_lon = max(coords['latitude'])

n = len(coords['path_id'].unique())

#each path_id has its own map and colour
for i in range(0,n-1):

  m = folium.Map()
  color_path = colors[i]
  gps_path = list(zip(path[i]['longitude'], path[i]['latitude']))
  folium.PolyLine(locations=gps_path, color=color_path, weight=2.5, opacity=0.8).add_to(m)
  folium.Marker(gps_path[20], icon=folium.Icon(color=color_path), popup=f'Safe Path').add_to(m)

  color_decoy_path = colors[i + n]
  gps_decoy_path = list(zip(decoy_path[i]['longitude'], decoy_path[i]['latitude']))
  folium.PolyLine(locations=gps_decoy_path, color=color_decoy_path, weight=2.5, opacity=0.8).add_to(m)
  folium.Marker(gps_decoy_path[10], icon=folium.Icon(color=color_decoy_path), popup=f'Decoy Path').add_to(m)

  m.fit_bounds([(min_lat, min_lon), (max_lat, max_lon)])
  maps.append(m)

  display(maps[i])


<ipython-input-258-e5ff1d155bdd>:24: UserWarning: color argument of Icon should be one of: {'darkred', 'purple', 'darkblue', 'darkgreen', 'blue', 'lightblue', 'lightred', 'red', 'cadetblue', 'gray', 'pink', 'orange', 'lightgray', 'white', 'darkpurple', 'beige', 'black', 'lightgreen', 'green'}.
  folium.Marker(gps_decoy_path[10], icon=folium.Icon(color=color_decoy_path), popup=f'Decoy Path').add_to(m)


<ipython-input-258-e5ff1d155bdd>:19: UserWarning: color argument of Icon should be one of: {'darkred', 'purple', 'darkblue', 'darkgreen', 'blue', 'lightblue', 'lightred', 'red', 'cadetblue', 'gray', 'pink', 'orange', 'lightgray', 'white', 'darkpurple', 'beige', 'black', 'lightgreen', 'green'}.
  folium.Marker(gps_path[20], icon=folium.Icon(color=color_path), popup=f'Safe Path').add_to(m)


In [259]:
def distance(current_point, coords_path):
  nearest_point_on_line = min(coords_path, key=lambda point: geodesic(point, current_point).kilometers)
  return geodesic(current_point, nearest_point_on_line).kilometers

In [260]:
import folium
import random

maps = []

colors = ['red','blue','green','yellow','orange','purple','pink','cyan','black','brown','maroon','navy','teal','olive','lime','aqua','silver','gold']

max_lat = max(coords['longitude'])
min_lat = min(coords['longitude'])
min_lon = min(coords['latitude'])
max_lon = max(coords['latitude'])

n = len(coords['path_id'].unique())

for i in range(0,n-1):
    m = folium.Map()
    gps_path = list(zip(path[i]['longitude'], path[i]['latitude']))
    folium.PolyLine(locations=gps_path, color=colors[i], weight=2.5, opacity=0.8).add_to(m)

    gps_decoy_path = list(zip(decoy_path[i]['longitude'], decoy_path[i]['latitude']))
    folium.PolyLine(locations=gps_decoy_path, color=colors[i+n], weight=2.5, opacity=0.8).add_to(m)

    for index in range(0, len(decoy_path[i]), 5):
      current_point = (decoy_path[i].iloc[index]['latitude'], decoy_path[i].iloc[index]['longitude'])
      perp_dist = distance(current_point, gps_path)
      if perp_dist < 0.02:
          colour = 'green'
      elif perp_dist >= 0.04:
          colour = 'red'
      else:
          colour = 'orange'
      folium.Marker(location=current_point,
                    popup=f"Deviation: {perp_dist:.2f} km",
                    icon=folium.Icon(color=colour)).add_to(m)

    m.fit_bounds([(min_lat, min_lon), (max_lat, max_lon)])
    maps.append(m)
    display(maps[i])